In [25]:
import pandas as pd

df = pd.read_parquet("sih_rs_tratado.parquet")

In [32]:
import polars as pl
from pathlib import Path

# --- Configuração ---
# Ajuste o caminho se necessário
PROCESSED_FILE_PATH = Path("sih_rs_tratado.parquet")

def analisar_frequencia_etnia(caminho_arquivo: Path):
    """
    Lê um arquivo Parquet e conta a frequência dos 10 valores mais comuns
    na coluna 'ETNIA'.
    """
    if not caminho_arquivo.exists():
        print(f"ERRO: Arquivo não encontrado em '{caminho_arquivo}'")
        return

    print(f"Analisando o arquivo: {caminho_arquivo.name}")
    print("-" * 40)

    try:
        # 1. Lê o arquivo, selecionando apenas a coluna 'ETNIA' para economizar memória
        df = pl.read_parquet(caminho_arquivo, columns=["QT_DIARIAS"])

        # 2. Conta a frequência de cada valor na coluna, ordena do mais frequente
        #    para o menos, e pega os 10 primeiros.
        contagem_etnia = (
            df.group_by("QT_DIARIAS")
            .agg(pl.count().alias("contagem")) # Conta as ocorrências e renomeia para 'contagem'
            .sort("contagem", descending=True) # Ordena pela contagem
            .limit(10)                         # Pega os 10 primeiros
        )

        print("Contagem dos 10 valores mais frequentes para a coluna 'ETNIA':")
        print(contagem_etnia)

    except pl.ColumnNotFoundError:
        print("ERRO: A coluna 'ETNIA' não foi encontrada no arquivo.")
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")

# --- Execução ---
if __name__ == "__main__":
    analisar_frequencia_etnia(PROCESSED_FILE_PATH)



Analisando o arquivo: sih_rs_tratado.parquet
----------------------------------------
Contagem dos 10 valores mais frequentes para a coluna 'ETNIA':
shape: (10, 2)
┌────────────┬──────────┐
│ QT_DIARIAS ┆ contagem │
│ ---        ┆ ---      │
│ i32        ┆ u32      │
╞════════════╪══════════╡
│ 0          ┆ 11820567 │
│ 200        ┆ 139      │
│ 214        ┆ 107      │
│ 216        ┆ 91       │
│ 204        ┆ 73       │
│ 224        ┆ 67       │
│ 210        ┆ 64       │
│ 212        ┆ 61       │
│ 206        ┆ 57       │
│ 208        ┆ 56       │
└────────────┴──────────┘


/tmp/ipykernel_17885/178973963.py:28: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  .agg(pl.count().alias("contagem")) # Conta as ocorrências e renomeia para 'contagem'


In [ ]:
import pandas as pd
from pathlib import Path

# Define the file path. Assumes the notebook is in the 'interim' folder.
file_path = Path("sih_rs_tratado.parquet")

if not file_path.exists():
    print(f"Error: The file '{file_path}' was not found. Make sure the pipeline's step 3 has been run and the notebook is in the correct folder.")
else:
    try:
        # Read the entire Parquet file into a pandas DataFrame.
        # This will show all columns that exist in the file.
        df = pd.read_parquet(file_path)

        # Show all existing columns to confirm if DIAS_CALCULADOS was created.
        print("All columns found in the file:")
        print(df.columns.tolist())

        # Now, try to select the relevant columns.
        if "DIAS_CALCULADOS" in df.columns and "DIAS_PERM" in df.columns:
            df_selected = df[["N_AIH", "DT_INTER", "DT_SAIDA", "DIAS_PERM", "DIAS_CALCULADOS"]]
            
            print("\nComparing DIAS_PERM and DIAS_CALCULADOS:")
            # Show a sample of records where the values are different.
            df_diff = df_selected[df_selected["DIAS_PERM"] != df_selected["DIAS_CALCULADOS"]].head(20)
            print(df_diff)

            # Print a summary of the differences.
            total_diff = len(df_selected[df_selected["DIAS_PERM"] != df_selected["DIAS_CALCULADOS"]])
            total_records = len(df_selected)
            print(f"\nTotal records checked: {total_records:,}")
            print(f"Total records with a difference: {total_diff:,}")
        else:
            print("\nError: The required columns 'DIAS_CALCULADOS' or 'DIAS_PERM' were not found in the file. Please check your preprocess script.")

    except Exception as e:
        print(f"An error occurred while reading the file: {e}")

ColumnNotFoundError: DIAS_CALCULADOS

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'sink' <---
Parquet SCAN [sih_rs_tratado.parquet] [id: 124728243191920]
PROJECT */58 COLUMNS